#### The portfolio strategies for now include:

    - Minimum Variance
        you can specify the number of periods to consider for the rolling window of covariance matrices (var_window), to estimate next period volatility.

    - Equally weighted
        you can specify when the rebalancing will happen (from x to x periods).

    - Sharpe Ratio Maximization in the Sample Period
        you can specify the number of periods to consider for the rolling window of covariance matrices (var_window), to estimate next period volatility. You can also specify a minimum period to consider to estimate the next period HPR (This is accomplished via an expanding window min_per). This function assumes that we want the closest previous var_window periods to predict volatility because this has been shown to be more effective due to volatility clustering, but regarding the prediction of HPR, we want to use the longest time prediction possible.
        

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import yfinance as yf

In [1]:
def MinimizePortfolioVariance(CovarReturns, RiskFreeRate, PortfolioSize):
    
    # define maximization of Sharpe Ratio using principle of duality
    def  f(x, CovarReturns, RiskFreeRate, PortfolioSize):
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) ) #Portfolio stdev is sqrt(((weights @ covmat) @ weights.T))
        func = funcDenomr
        return func

    #define equality constraint representing fully invested portfolio
    def constraintEq(x):
        A=np.ones(x.shape)
        b=1 #the sum of the weights will be equal to 1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize) #will start with weights being 0.33
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0 #no shortsale constraint
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])
    
    #invoke minimize solver
    opt = optimize.minimize (f, x0 = xinit, args = (CovarReturns,\
                             RiskFreeRate, PortfolioSize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)

    return opt


def MinimumVarianceWeights(PriceMatrix, var_window=12,Rf=0):

    HPR_matrix = (PriceMatrix / PriceMatrix.shift(1) - 1).dropna(how='all') - Rf # creating the Holding period excess returns matrix. IMPORTANT, Rf must be in the same measure as returns. for exemple, if returns are daily, and rf is 1% yearly, you should place 0.01/252, or just ignore
    Weights=pd.DataFrame(columns=PriceMatrix.columns,index=PriceMatrix.index[var_window:]) # creating an empty weights dataframe

    for k in tqdm(range(len(HPR_matrix)-var_window)): 
        StocksWOnan=HPR_matrix.iloc[k:k+var_window,:].dropna(axis=1).columns #The StocksWOnan will make the function robust to periods where Stocks do not have prices
        PortfolioSize=len(StocksWOnan) #the number of columns with prices is the number of stocks we can invest in our portfolio
        CovMat=np.array(HPR_matrix[StocksWOnan].iloc[k:k+var_window,:].astype(float).cov()) #get the covariance matrix of the excess returns    
        xOptimal =[]  
        result = MinimizePortfolioVariance(CovMat, Rf, PortfolioSize) #minimize the portfolio variance
        xOptimal.append(result.x) 
        xOptimalArray = np.array(xOptimal)
        
        Weights.loc[HPR_matrix.index[k+var_window],StocksWOnan] = xOptimalArray
        
    
    return Weights



def EquallyWeightedPortfolio(PriceMatrix,rebalancing_period=20):

    Weights=pd.DataFrame(columns=PriceMatrix.columns,index=PriceMatrix.index)
    StocksWOnan=PriceMatrix.iloc[0,:].dropna().index # stocks without nan at the start of the data
    portfolioSize=len(StocksWOnan) #number of stocks to equally weigh at the beginning

    for k in tqdm(range(len(PriceMatrix))):
        if k % rebalancing_period == 0: # If we are in a rebalancing period
            StocksWOnan=PriceMatrix.iloc[k,:].dropna().index #stocks without nan in the analysed rebalancing period
            portfolioSize=len(StocksWOnan)
            Weights.loc[PriceMatrix.index[k],StocksWOnan]= 1/portfolioSize
        else: #if we are not in a rebalancing period
            Weights.loc[PriceMatrix.index[k],StocksWOnan]= 1/portfolioSize
    
    return Weights




def MaximizeSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
    # define maximization of Sharpe Ratio using principle of duality
    def  f(x, MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) )
        funcNumer = np.matmul(np.array(MeanReturns),x.T)-RiskFreeRate
        func = -(funcNumer / funcDenomr)
        return func

    #define equality constraint representing fully invested portfolio
    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])
    
    #invoke minimize solver
    opt = optimize.minimize (f, x0 = xinit, args = (MeanReturns, CovarReturns,\
                             RiskFreeRate, PortfolioSize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)    
    return opt


def MaxSharpeRatioPortfolio(PriceMatrix,var_window=12,min_per=10,Rf=0):
    HPR_matrix = (PriceMatrix / PriceMatrix.shift(1) - 1).dropna(how='all') - Rf # creating the Holding period excess returns matrix. IMPORTANT, Rf must be in the same measure as returns. for exemple, if returns are daily, and rf is 1% yearly, you should place 0.01/252, or just ignore
    ExpectRet = HPR_matrix.expanding(min_periods=min_per).mean().shift(1).dropna(how='all')  #The expected return uses the average hpr observed since inception, we use shift(1) to not include the hpr for t as the expected return for the same period t
    HPR_matrix = HPR_matrix.iloc[min_per:,:]
    Weights=pd.DataFrame(columns=HPR_matrix.columns,index=HPR_matrix.index) # creating an empty weights dataframe

    for k in tqdm(range(len(HPR_matrix)-var_window)):
        
        s1=HPR_matrix.iloc[k:k+var_window,:].dropna(axis=1).columns
        s2=ExpectRet.iloc[k:k+var_window,:].dropna(axis=1).columns
        StocksWOnan = s1.intersection(s2)
        portfolioSize=len(StocksWOnan)
        CovMat=np.array(HPR_matrix[StocksWOnan].iloc[k:k+var_window,:].astype(float).cov())
        ExcessRet=ExpectRet[StocksWOnan].iloc[k+var_window,:] #This is the expected excess returns, we are going to use to compute tomorrow expected returns                
    #OPTIMAL SHARPE RATIO

        xOptimal = []
        result = MaximizeSharpeRatioOptmzn(ExcessRet, CovMat, Rf, portfolioSize)
        xOptimal.append(result.x)        
        xOptimalArray = np.array(xOptimal)        
        Weights.loc[ExpectRet.index[k+var_window],StocksWOnan] = xOptimalArray
    
    return Weights.dropna(how='all')

# Showing an example

In [3]:
def getPrices(tickers,start_date,end_date):
    Prices = pd.DataFrame()
    for i in tickers:
        Prices = pd.concat([Prices,yf.download(i, start=start_date, end=end_date)['Close']],axis=1)
    Prices.columns = tickers

    return Prices

# Set the start and end dates for the price data
start_date = '2010-01-01'  # Replace with your desired start date
end_date = '2022-01-01'  # Replace with your desired end date
tickers = ['AMZN','MSFT','TSLA','GOOG','META','JPM','NVDA']

PricesMatrix = getPrices(tickers,start_date,end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
PricesMatrix

,AMZN,MSFT,TSLA,GOOG,META,JPM,NVDA
2010-01-04,6.695000,30.950001,NaN,15.610239,NaN,42.849998,4.622500
2010-01-05,6.734500,30.959999,NaN,15.541497,NaN,43.680000,4.690000
2010-01-06,6.612500,30.770000,NaN,15.149715,NaN,43.919998,4.720000
2010-01-07,6.500000,30.450001,NaN,14.797037,NaN,44.790001,4.627500
2010-01-08,6.676000,30.660000,NaN,14.994298,NaN,44.680000,4.637500
...,...,...,...,...,...,...,...
2021-12-27,169.669495,342.450012,364.646667,148.063995,346.179993,158.160004,309.450012
2021-12-28,170.660995,341.250000,362.823334,146.447998,346.220001,158.639999,303.220001
2021-12-29,169.201004,341.950012,362.063324,146.504501,342.940002,158.559998,300.010010
2021-12-30,168.644501,339.320007,356.779999,146.002502,344.359985,158.479996,295.859985


In [ ]:
MinVarianceWeights = MinimumVarianceWeights(PricesMatrix, window=10,Rf=0) #The window parameter gives the nº of periods to use the covariance matrix
MinVarianceWeights

In [ ]:
EquallyWeightedWeights = EquallyWeightedPortfolio(PricesMatrix,rebalancing_period=20) #The portfolio will be rebalanced each rebalancing_periods.if daily data, and rebalancing_periods=20, then the weights will be re adjusted every 20 days
EquallyWeightedWeights

In [16]:
MaxSharpeRatioWeights = MaxSharpeRatioPortfolio(PricesMatrix,var_window=12,min_per=10,Rf=0)
MaxSharpeRatioWeights

100%|██████████| 2998/2998 [00:18<00:00, 159.51it/s]


,AMZN,MSFT,TSLA,GOOG,META,JPM,NVDA
2010-02-05,0.0,0.0,NaN,0.0,NaN,0.408449,0.591551
2010-02-08,0.0,0.0,NaN,0.0,NaN,0.0,1.0
2010-02-09,0.0,0.0,NaN,0.0,NaN,0.000939,0.999061
2010-02-10,0.0,0.0,NaN,0.0,NaN,0.010773,0.989227
2010-02-11,0.0,0.0,NaN,0.0,NaN,0.0,1.0
...,...,...,...,...,...,...,...
2021-12-27,0.267256,0.0,0.0,0.179561,0.226001,0.327181,0.0
2021-12-28,0.310501,0.0,0.0,0.192707,0.20237,0.294421,0.0
2021-12-29,0.327054,0.0,0.0,0.16956,0.204947,0.298439,0.0
2021-12-30,0.346938,0.0,0.0,0.170409,0.165037,0.317616,0.0
